In [1]:
# Cell 1: 导入依赖和初始设置
import gc
import os
import random
import warnings
import joblib
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from utils import Logger, evaluate, gen_sub

warnings.filterwarnings('ignore')
seed = 2020
random.seed(seed)

In [2]:
# Cell 2: 参数配置和日志记录
mode = 'valid'  # 'valid' or 'online'
os.makedirs('../user_data/log', exist_ok=True)
log = Logger('../user_data/log/notebook.log').logger
log.info(f'lightgbm 排序，mode: {mode}')


2025-03-19 22:18:27,215 - C:\Users\76276\AppData\Local\Temp\ipykernel_24720\2413840883.py[line:5] - INFO: lightgbm 排序，mode: valid


In [3]:
# Cell 3: 定义模型训练函数
def train_model(df_feature, df_query):
    df_train = df_feature[df_feature['label'].notnull()]
    df_test = df_feature[df_feature['label'].isnull()]
    
    del df_feature
    gc.collect()
    
    ycol = 'label'
    feature_names = list(filter(lambda x: x not in [ycol, 'created_at_datetime', 'click_datetime'], df_train.columns))
    feature_names.sort()
    
    model = lgb.LGBMClassifier(
        num_leaves=64,
        max_depth=10,
        learning_rate=0.05,
        n_estimators=10000,
        subsample=0.8,
        feature_fraction=0.8,
        reg_alpha=0.5,
        reg_lambda=0.5,
        random_state=seed,
        importance_type='gain',
        metric=None,
        device_type='gpu',
        gpu_device_id=0,
        gpu_use_dp=True,
        boost_from_average=True
    )
    
    oof = []
    prediction = df_test[['user_id', 'article_id']]
    prediction['pred'] = 0
    df_importance_list = []
    
    kfold = GroupKFold(n_splits=5)
    for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(df_train[feature_names], df_train[ycol], df_train['user_id'])):
        X_train = df_train.iloc[trn_idx][feature_names]
        Y_train = df_train.iloc[trn_idx][ycol]
        X_val = df_train.iloc[val_idx][feature_names]
        Y_val = df_train.iloc[val_idx][ycol]
        
        callbacks = [
            lgb.log_evaluation(period=100),
            lgb.early_stopping(stopping_rounds=100)
        ]
        
        lgb_model = model.fit(
            X_train, Y_train,
            eval_set=[(X_train, Y_train), (X_val, Y_val)],
            eval_names=['train', 'valid'],
            eval_metric='auc',
            callbacks=callbacks
        )
        
        pred_val = lgb_model.predict_proba(X_val, num_iteration=lgb_model.best_iteration_)[:, 1]
        df_oof = df_train.iloc[val_idx][['user_id', 'article_id', ycol]].copy()
        df_oof['pred'] = pred_val
        oof.append(df_oof)
        
        pred_test = lgb_model.predict_proba(df_test[feature_names], num_iteration=lgb_model.best_iteration_)[:, 1]
        prediction['pred'] += pred_test / 5
        
        df_importance = pd.DataFrame({
            'feature_name': feature_names,
            'importance': lgb_model.feature_importances_,
        })
        df_importance_list.append(df_importance)
        
        joblib.dump(model, f'../user_data/model/lgb{fold_id}.pkl')
    
    return prediction, oof, df_importance_list

In [4]:
# Cell 4: 定义在线预测函数
def online_predict(df_test):
    ycol = 'label'
    feature_names = list(filter(lambda x: x not in [ycol, 'created_at_datetime', 'click_datetime'], df_test.columns))
    feature_names.sort()
    
    prediction = df_test[['user_id', 'article_id']]
    prediction['pred'] = 0
    
    for fold_id in tqdm(range(5)):
        model = joblib.load(f'../user_data/model/lgb{fold_id}.pkl')
        pred_test = model.predict_proba(df_test[feature_names])[:, 1]
        prediction['pred'] += pred_test / 5
    
    return prediction

In [5]:
# Cell 5: 数据加载与预处理
def load_and_process_data(mode='valid'):
    if mode == 'valid':
        df_feature = pd.read_pickle('../user_data/data/offline/feature.pkl')
        df_query = pd.read_pickle('../user_data/data/offline/query.pkl')
        
        for f in df_feature.select_dtypes('object').columns:
            lbl = LabelEncoder()
            df_feature[f] = lbl.fit_transform(df_feature[f].astype(str))
            
        prediction, oof, importance_list = train_model(df_feature, df_query)
        return prediction, oof, importance_list, df_query
    else:
        df_feature = pd.read_pickle('../user_data/data/online/feature.pkl')
        prediction = online_predict(df_feature)
        return prediction

In [6]:
# Cell 6: 生成提交文件(可单独运行)
def generate_submission(prediction):
    df_sub = gen_sub(prediction)
    df_sub.sort_values(['user_id'], inplace=True)
    os.makedirs('../prediction_result', exist_ok=True)
    df_sub.to_csv('../prediction_result/result.csv', index=False)
    return df_sub

In [7]:

# Cell 7: 执行主流程
if mode == 'valid':
    prediction, oof, importance_list, df_query = load_and_process_data(mode='valid')
    # 评估指标计算
    df_oof = pd.concat(oof)
    total = df_query[df_query['click_article_id'] != -1].user_id.nunique()
    metrics = evaluate(df_oof, total)
    print("评估指标:", metrics)
else:
    prediction = load_and_process_data(mode='online')



[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Info] Number of positive: 25400, number of negative: 3874573
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 6674
[LightGBM] [Info] Number of data points in the train set: 3899973, number of used features: 28
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 4060 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 16
[LightGBM] [Info] 28 dense feature groups (104.14 MB) transferred to GPU in 0.090637 secs. 0 sparse feature groups
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Info] [b

100%|██████████| 31750/31750 [00:06<00:00, 4934.23it/s]

评估指标: (0.37115604351930054, 0.22491968470034504, 0.47600476923841223, 0.23915152031303147, 0.5645585970490338, 0.24536212270548632, 0.6474241144617219, 0.24829571570413186, 0.6739033235630185, 0.24888172977754455)


In [8]:
# Cell 8: 生成提交文件(按需运行)
df_submission = generate_submission(prediction)
df_submission.head()

100%|██████████| 50000/50000 [03:45<00:00, 222.17it/s]


,user_id,article_1,article_2,article_3,article_4,article_5
49999,200000,194935,336221,195087,195645,59681
49998,200001,272143,64329,199198,324823,166581
49997,200002,202701,208596,205982,203288,206711
49996,200003,277107,158772,235105,50494,156807
49995,200004,218028,289003,157478,57966,202355
